In [1]:
! python -m spacy download es_core_news_md

c:\PROGRAMAS\python\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\PROGRAMAS\python\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\PROGRAMAS\python\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 42.3/42.3 MB 6.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


<b>Interesa que el modelo sea capaz de detectar las siguientes entidades:<b> <br>

LEY ORGANICA -> la Ley ORGANICA 8/1980 <br>
REAL DECRETO -> Real Decreto 357/1991<br>
RESOLUCION -> Resolucion de 24 de agosto de 2015<br>
BOE -> BOE-A-2015-11430<br>
ARTICULOS -> el art. 90.2 y 3<br>
LEY ->  Ley del Estatuto de los Trabajadores<br>
FECHAS -> 24 de agosto de 2015, 23 de octubre
DISPOSICIONES -> Disposicion adicional cuarta, disposicion adicional 8<br>

In [2]:

import spacy
import json
cv_data = json.load(open('annotations.json', 'r', encoding="utf8"))

print(cv_data)
print(len(cv_data))

c:\PROGRAMAS\python\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\PROGRAMAS\python\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\PROGRAMAS\python\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


[['BOE-A-1987-15276 la Ley Orgánica 3/1987, de 2 de julio\r\n', {'entities': [[0, 16, 'BOE'], [20, 39, 'LEY'], [44, 54, 'FECHA']]}], ['\r\nBOE-A-2004-4456 arts. 9.3.c) y 28.2.d) de la Ley del Impuesto sobre Sociedades, texto refundido aprobado por Real Decreto Legislativo 4/2004, de 5 de marzo\r\n', {'entities': [[2, 17, 'BOE'], [18, 40, 'ART'], [111, 142, 'REALDE'], [147, 157, 'FECHA']]}], ['\r\nBOE-A-1993-25359 art. 45.1 A) de la Ley del Impuesto sobre Transmisiones Patrimoniales, texto refundido aprobado por Real Decreto Legislativo 1/1993, de 24 de septiembre\r\n', {'entities': [[2, 18, 'BOE'], [19, 31, 'ART'], [119, 150, 'REALDE'], [155, 171, 'FECHA']]}], ['\r\nBOE-A-1992-28740 art. 20.1 de la Ley 37/1992, de 28 de diciembre\r\n', {'entities': [[2, 18, 'BOE'], [19, 28, 'ART'], [35, 46, 'LEY'], [51, 66, 'FECHA']]}], ['\r\nBOE-A-2006-20764 un art. 61 bis a la Ley 35/2006, de 28 de noviembre\r\n', {'entities': [[2, 18, 'BOE'], [22, 33, 'ART'], [39, 50, 'LEY'], [55, 70, 'FECHA']]}], [

In [3]:
from spacy.tokens import DocBin
from tqdm import tqdm

def get_spacy_doc(file, data):
    # Create a blank spaCy pipeline
    nlp = spacy.blank('es')
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = []

        # Extract entities from the annotations
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            entity_indices = entity_indices + list(range(start, end))
            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                # Log errors for annotations that couldn't be processed
                err_data = str([start, end]) + "    " + str(text) + "\n"
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db

In [4]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(cv_data, test_size=1/3)
print(len(train), len(test))

# Open a file to log errors during annotation processing
file = open('train_file.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

# Close the error log file
file.close()

110 55


100%|██████████| 55/55 [00:00<00:00, 3687.51it/s]


In [5]:
spacy.require_cpu()

!python -m spacy train config.cfg  --output ./trained_models/output  --paths.train train_data.spacy  --paths.dev test_data.spacy

ℹ Saving to output directory: trained_models\output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     46.46    8.21    4.91   25.12    0.08
  7     200         66.29   1982.80   96.87   96.63   97.10    0.97
 17     400         25.90    171.92   96.10   97.04   95.17    0.96
 29     600         21.87     92.75   96.39   96.15   96.62    0.96
 43     800         28.74     77.03   95.86   96.57   95.17    0.96
 61    1000          6.60     10.02   96.35   97.06   95.65    0.96
 83    1200          0.00      0.00   96.35   97.06   95.65    0.96
110    1400          0.00      0.00   96.35   97.06   95.65    0.96
142    1600     

c:\PROGRAMAS\python\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\PROGRAMAS\python\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\PROGRAMAS\python\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [6]:
# Load the trained spaCy NER model from the specified path
nlp = spacy.load('./trained_models/output/model-best')

<b> Evaluación del modelo <b>

In [8]:
import json
from spacy.scorer import Scorer
from spacy.training.example import Example

model = nlp

with open('model_evaluation_file_annotations.json', 'r') as file:
    data = json.load(file)

file.close()

def evaluate_my_ner_model(model, data):

    scorer = Scorer()
    examples = []

    for text, annots in data:
        doc = model.make_doc(text)
        examples.append(Example.from_dict(doc, annots))

    return model.evaluate(examples)

results = evaluate_my_ner_model(model, data)


print("La precisión del modelo es {}".format(results['ents_p']))
print("El recall del modelo es {}".format(results['ents_r']))
print("El Fscore del modelo es {}".format(results['ents_f']))

La precisión del modelo es 0.9705882352941176
El recall del modelo es 0.9611650485436893
El Fscore del modelo es 0.9658536585365853


In [9]:
from spacy import displacy

colors = {'BOE': '#ff0000',
          'FECHA':'#5564eb',
          'REALDE': '#00825a',
          'LEY': '#ffea00',
          'ART': '#7d2181',
          'DISP': '#ffc0cb'}

options = {'ents': ['BOE', 'FECHA', 'REALDE', 'LEY', 'ART', 'DISP'], 'colors':colors}

texts = ['BOE-A-1985-11672 arts. 187.2, 201.3, disposición adicional 1.2 y AÑADE el art. 44 bis y la disposición transitoria 7 a la Ley Orgánica 5/1985, de 19 de junio',
         'BOE-A-1985-12666 arts. 109.1, 110.3, 122.1, 310, 348, 356, 357, 358.2, 373, 433 bis, 434.2, AÑADE los arts. 136 bis y 360 bis y SUPRIME el 370 de la Ley Orgánica 6/1985, de 1 de julio',
         'BOE-A-1982-837 el art. 14.1 de la Ley 50/1981, de 30 de diciembre',
         'BOE-A-2000-323 los arts. 188.1, 217, AÑADE el 11 bis y TRANSPONE la Directiva 97/80 a la Ley 1/2000, de 7 de enero',
         'BOE-A-1998-16718 arts. 19.1, 60 y TRASPONE la Directiva 97/80 a la Ley 29/1998, de 13 de julio',
         'BOE-A-1994-16224 art. 16.1 de la Ley 25/1994, de 12 de julio',
         'BOE-A-1986-10499 arts. 3, 6, 18 y 21.1 de la Ley 14/1986, de 25 de abril',
         'BOE-A-2003-10715 arts. 2, 11.2, 12.2, 34, 44, 53 y 63 de la Ley 16/2003, de 28 de mayo',
         'BOE-A-1995-7730 arts. 4.2, 17, 34, 37, 38.3, 45.1, 46, 48, 53.4, 54.2, 55.5, 64, 85, 90 y AÑADE el 48 bis y las disposiciones adicionales 17 y 18 a la Ley del Estatuto de los Trabajadores, texto refundido aprobado por Real Decreto Legislativo 1/1995, de 24 de marzo',
         'BOE-A-1995-24292 arts. 5 y 26 de la Ley 31/1995. de 8 de noviembre',
         'BOE-A-1995-8758 arts. 27.2, 108.2, 122.2, 146, 149.2, 180.1 y 181 de la Ley de Procedimiento Laboral, texto refundido aprobado por Real Decreto Legislativo 2/1995, de 7 de abril',
         'BOE-A-2000-15060 arts. 7, 8 y 46 y AÑADE la subsección 3 bis en la sección 2 del capítulo VI a la Ley de Infracciones y Sanciones en el Orden Social, texto refundido aprobado por Real Decreto Legislativo 5/2000, de 4 de agosto',
         'BOE-A-1998-20925 art. 1 del Real Decreto-ley 11/1998, de 4 de septiembre',
         'BOE-A-2001-13265 Disposición adicional 2 de la Ley 12/2001, de 9 de julio',
         'BOE-A-1994-14960 determinados preceptos; renumera el capítulo IV ter como IV quater; AÑADE los capítulos IV ter, IV quinquies y la disposición adicional 44 a la Ley General de la Seguridad Social, texto refundido aprobado por Real Decreto Legislativo 1/1994, de 20 de junio',
         'BOE-A-1984-17387 arts. 29 y 30  de la Ley 30/1984, de 2 de agosto',
         'BOE-A-1999-11194 arts. 108.2, 112, 132, 141.1e) y 148 de la Ley 17/1999, de 18 de mayo',
         'BOE-A-1964-2140 art. 69.3 de la Ley de Funcionarios Civiles del Estado, texto articulado aprobado por Decreto 315/1964, de 7 de febrero',
         'BOE-A-2003-23101 arts. 59.3 y 61.2 de la Ley 55/2003, de 16 de diciembre',
         'BOE-A-2000-12140 arts. 12.b) y 22 de la Ley sobre Seguridad Social de los Funcionarios Civiles del Estado, texto refundido aprobado por Real Decreto Legislativo 4/2000, de 23 de junio',
         'BOE-A-1999-22670 arts. 56.2, 60, 75 y 83.1 de la Ley 42/1999, de 25 de noviembre',
         'BOE-A-1984-16737 art. 34.10 de la Ley 26/1984, de 19 de julio',
         'BOE-A-1989-30361 art. 200 de la Ley de Sociedades Anónimas, texto refundido aprobado por Real Decreto Legislativo 1564/1989, de 22 de diciembre',
         'BOE-A-1977-12678 art. 1 y la disposición transitoria 1 de la Ley 36/1977, de 23 de mayo',
         'BOE-A-2003-23102 art. 22 bis a la Ley 56/2003, de 16 de diciembre',
         'BOE-A-1983-28126 art. 2 bis a la Ley 16/1983, de 24 de octubre',
         'BOE-A-1984-7250 Disposición adicional 3 a la Ley 5/1984, de 26 de marzo',
         'DOUE-L-2002-81758 Directiva 2002/73/CE, de 23 de septiembre',
         'DOUE-L-1998-80091 Directiva 97/80/CE, de 15 de diciembre',
         'DOUE-L-2004-82937 Directiva 2004/113/CE, de 13 de diciembre',
         'BOE-A-2022-14630, con efectos desde el 7 de octubre de 2022, los arts. 37.1.d), 38.1.d), 48, 66 y 68, por Ley Orgánica 10/2022, de 6 de septiembre',
         'BOE-A-2019-3244 los arts. 45.2, 46 y SE AÑADE la disposición transitoria 12, por Real Decreto-ley 6/2019, de 1 de marzo',
         'BOE-A-2013-8187 el art. 71.1  y SE SUPRIME la disposición transitoria 5, por Ley 11/2013, de 26 de julio',
         'BOE-A-2024-15936, con efectos desde el 22 de agosto de 2024, una disposición adicional, por Ley Orgánica 2/2024, de 1 de agosto',
         'BOE-A-2009-15958 la disposición transitoria 9, con efectos de 1 de enero de 2013, por Ley 9/2009, de 6 de octubre',
         'BOE-A-2024-6273 y se aprueba el Protocolo de actuación frente al acoso sexual y al acoso por razón de sexo en el ámbito de la Administración General del Estado y de sus organismos públicos: Real Decreto 247/2024, de 8 de marzo de 2024']

for text in texts:
  doc = nlp(text)
  #print(doc.ents)
  displacy.render(doc, style='ent', options=options)